# Imports

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings

from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import RobustScaler

# Configurações

In [3]:
data_path = Path(r'..\..\01_dados\01_dados_analise\01_dados_anomalia.csv')

# Funções e Classes de objetos

In [4]:
def perform_permutation_importance(modelo, X_train, X_test, y_train, y_test):
    """
    Realiza uma análise de importância da permutação em um modelo de machine learning fornecido.

    Esta função treina o modelo fornecido usando o conjunto de treino e, em seguida, calcula a
    importância da permutação para cada característica no conjunto de teste. Um gráfico de barras
    é gerado para visualizar a importância de cada característica.

    Parâmetros:
        modelo (estimator): O modelo de machine learning para analisar.
        X_train (array-like): Conjunto de dados de treino.
        X_test (array-like): Conjunto de dados de teste.
        y_train (array-like): Rótulos/targets do conjunto de dados de treino.
        y_test (array-like): Rótulos/targets do conjunto de dados de teste.

    Retorna:
        None: A função não retorna nenhum valor. Um gráfico de barras é exibido como saída.
    
    Exemplo:
        perform_permutation_importance(modelo, X_train, X_test, y_train, y_test)

    Nota:
        Certifique-se de que os dados de entrada estão devidamente preparados e limpos antes de usar
        esta função. Erros podem ocorrer se os dados contiverem valores nulos ou inf.
    """

    from sklearn.inspection import permutation_importance
    import matplotlib.pyplot as plt

    # Treinando o modelo
    modelo.fit(X_train, y_train)
    # Calculando a importância da permutação
    result = permutation_importance(modelo, X_test, y_test, n_repeats=30, random_state=0)
    # Plotando a importância da permutação
    sorted_idx = result.importances_mean.argsort()
    plt.barh(X_test.columns[sorted_idx], result.importances_mean[sorted_idx])
    plt.xlabel("Importância da Permutação")
    plt.show()

# Importação e preparação dos dados

In [5]:
df = pd.read_csv(data_path)
df.head()

,id_subsistema,din_instante,"Geração no Centro de Gravidade - MW médios (Gp,j) - MWh","Fator de Abatimento de Perdas Internas Instantâneas (F_PDIp,j)","Fator de Rateio das Perdas de Geração (UXP_GLFp,j)*","Deslocamento Hidráulico Energético Preliminar (DH_ENER_PRE_UHp,j)","Garantia física modulada ajustada pelo fator de disponibilidade (GFIS_2p,j)","Garantia Física de Repasse de Risco Hidrológico Modulada e Ajustada (GFIS_2_RRHp,j) - MWh","Garantis Física Modulada Ajustada de Repasse do Risco Hidrológico (GFIS_3_RRHp,j) - MWh","Fator de Risco Hidrológico aceito pelo gerador, variando entre 0 e 11% (Fp,j)",...,val_verifconstrainedoff,val_importacaoprogramada,val_importacaodespachada,val_importacaoverificada,val_preco_importacao,val_dispf,val_indisppf,val_indispff,val_intercambiomwmed,anomaly
0,N,2018-01-06 00:00:00,8564.245664,18.764152,18.688987,108.091180,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,S,2018-01-06 00:00:00,7715.282975,213.306544,213.352056,89.507131,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,NE,2018-01-06 00:00:00,2825.618028,19.924679,19.714905,68.291389,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,SE,2018-01-06 00:00:00,30843.187134,373.242309,374.237593,371.119404,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,SE,2018-01-06 01:00:00,30513.088050,372.459187,374.129311,327.623405,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [6]:
# colocando o cmo no final do dataframe para facilitar a manipulação dos dados
cols = [col for col in df if col != 'cmo'] + ['cmo']
df = df[cols]

In [12]:
df.din_instante

0        2018-01-06 00:00:00
1        2018-01-06 00:00:00
2        2018-01-06 00:00:00
3        2018-01-06 00:00:00
4        2018-01-06 01:00:00
                ...         
81019    2022-05-31 22:00:00
81020    2022-05-31 23:00:00
81021    2022-05-31 23:00:00
81022    2022-05-31 23:00:00
81023    2022-05-31 23:00:00
Name: din_instante, Length: 81024, dtype: object

In [7]:
df_aux = df.copy()
df_aux.drop(['id_subsistema', 'din_instante'], inplace=True, axis=1)

In [ ]:
# Separando as variáveis para modelagem X são as variáveis independentes e Y a Variável indepentende
X = df_aux.iloc[:, :-1].values 
Y = df_aux.iloc[:, -1].values 

scaler = RobustScaler()

X = scaler.fit_transform(X)

# Método 3: Importância de Permutação

A Importância de Permutação é um conceito utilizado para calcular a importância de cada variável de um modelo de machine learning. Ela funciona embaralhando os valores de uma única característica e medindo o quanto essa mudança afeta a performance do modelo. Isso é feito para cada característica, proporcionando um ranking de importância das variáveis que ajuda a entender quais são as mais relevantes para as previsões do modelo.

## Explicação para a função `perform_permutation_importance`

A função `perform_permutation_importance` realiza uma análise de importância de permutação em um modelo de machine learning fornecido. Esta análise ajuda a entender a influência de cada característica na predição do modelo, proporcionando uma visão mais clara e profunda do comportamento do modelo.

## Como funciona?

1. **Treinamento do Modelo:** 
   - O modelo fornecido é treinado usando os conjuntos de dados de treino (`X_train` e `y_train`).

2. **Cálculo da Importância de Permutação:** 
   - Após o treinamento, a função calcula a importância da permutação para cada característica no conjunto de teste (`X_test`). 

3. **Plot da Importância de Permutação:** 
   - A função então gera um gráfico de barras, visualizando a importância de cada característica.

## Parâmetros da Função:

- `modelo`: O modelo de machine learning para analisar.
- `X_train`: Conjunto de dados de treino.
- `X_test`: Conjunto de dados de teste.
- `y_train`: Rótulos/targets do conjunto de dados de treino.
- `y_test`: Rótulos/targets do conjunto de dados de teste.

In [ ]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

### Suport Vector Regressor

In [ ]:
modelo = SVR(kernel="linear")
perform_permutation_importance(modelo, X_train, X_test, y_train, y_test)

### Gradient Boosting Regressor

In [ ]:
modelo = GradientBoostingRegressor()
perform_permutation_importance(modelo, X_train, X_test, y_train, y_test)

### Decision Tree Regressor

In [ ]:
modelo = DecisionTreeRegressor()
perform_permutation_importance(modelo, X_train, X_test, y_train, y_test)

### XGB Regressor

In [ ]:
modelo = XGBRegressor(objective ='reg:squarederror')
perform_permutation_importance(modelo, X_train, X_test, y_train, y_test)

### RandomForestRegressor

In [ ]:
model = RandomForestRegressor()
perform_shap_analysis(model, X_train, X_test, y_train, y_test)

### Linear Regression

In [ ]:
model = LinearRegression()
perform_shap_analysis(model, X_train, X_test, y_train, y_test, model_type="linear")